<a href="https://colab.research.google.com/github/Qoonitah18/Project-1/blob/main/task_2_Concise%20Video%20Game%20Data%20Enhancement%20with%20Google%20AI%20Studio%20API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bagian 1 - Import, pengaturan file & konstanta

In [ ]:
import os
import time # jeda antar request
import re
import argparse
import random

import pandas as pd
# requests diperlukan ketika nanti mau pakai API real
import requests # untuk kirim HTTP (dipakai nanti jika pakai API sungguhan)

# Nama file input/output
INPUT_CSV = "Game Thumbnail.csv"
OUTPUT_CSV = "Game_Thumbnail_enhanced_api_sim.csv"


Bagian 2 - Template prompt & fungsi pemanggil API (mock + skeleton real)

Kita buat dua fungsi: call_api_mock(prompt, kind) untuk latihan, dan call_api_real(prompt) sebagai skeleton yang menggambarkan bagaimana panggilan ke Google AI Studio akan terlihat.

In [ ]:
# Prompt templates
# prompt ini yang bakalan dikirim ke AI. Pastiin output terstruktur
PROMPT_GENRE = 'Classify the genre of the game titled "{title}". Return exactly ONE word (e.g., Shooter, RPG, Puzzle).'
PROMPT_DESC  = 'Write a short description (max 30 words) for the game titled "{title}". Use present tense, no bullets.'
PROMPT_MODE  = 'For the game titled "{title}", return exactly one word: Singleplayer, Multiplayer, or Both.'

# fungsi latihan yang merespon seakan-akan AI, berdasarkan kata kunci di prompt
# berguna untuk testing alur
def call_api_mock(prompt: str, kind: str) -> str:

    # beri jeda kecil supaya mirip pemanggilan nyata
    time.sleep(0.15)
    t = prompt.lower()

    # return berdasarkan 'kind' (genre/desc/mode)
    if kind == "genre":
        if "valorant" in t or "shoot" in t or "hunt" in t:
            return "Shooter"
        if "civ" in t or "civilization" in t or "strategy" in t:
            return "Strategy"
        if "minecraft" in t or "sandbox" in t:
            return "Sandbox"
        if "hidden" in t or "manor" in t:
            return "Puzzle"
        return random.choice(["Casual", "Adventure", "RPG"])
    if kind == "desc":
        # contoh deskripsi singkat simulasi
        return "An engaging game that focuses on core mechanics and player progression."
    if kind == "mode":
        if "valorant" in t or "arena" in t:
            return "Multiplayer"
        if "minecraft" in t:
            return "Both"
        return "Singleplayer"

# contoh kode yang mengirim POST request ke endpoint
# struktur payload dan response bergantung pada dokumentasi Google AI Studio. Cth:{"instances":[{"input":...}]} atau sejenisnya
def call_api_real(prompt: str) -> str:
    """Kerangka panggilan API nyata. Jangan jalankan sebelum set API_KEY & ENDPOINT.
       Perlu disesuaikan dengan dokumentasi endpoint yang diberikan HR."""
    API_KEY = os.getenv("GOOGLE_API_KEY")
    ENDPOINT = os.getenv("GOOGLE_AI_ENDPOINT")
    if not API_KEY or not ENDPOINT:
        raise EnvironmentError("Set GOOGLE_API_KEY and GOOGLE_AI_ENDPOINT in your environment to use real API.")
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {"prompt": prompt}  # sesuaikan jika endpoint butuh 'instances' / 'inputs'
    r = requests.post(ENDPOINT, headers=headers, json=payload, timeout=15)
    r.raise_for_status()
    data = r.json()

    # JSON response shape bisa berbeda → kita ambil teks dengan cara aman:
    # contoh attempt: common keys -> "text", "output", "generated_text", "response"
    for key in ("text", "output", "generated_text", "response", "result"):
        if key in data and isinstance(data[key], str):
            return data[key].strip()

    # fallback: jika ada nested list/dict
    # ambil first string value yang masuk akal
    def find_str(obj):
        if isinstance(obj, str):
            return obj.strip()
        if isinstance(obj, dict):
            for v in obj.values():
                s = find_str(v)
                if s: return s
        if isinstance(obj, list):
            for v in obj:
                s = find_str(v)
                if s: return s
        return None
    found = find_str(data)
    return found or ""

Bagian 3 - Fungsi sanitasi (rapikan jawaban)

Pastikan jawaban yang masuk konsisten: genre satu kata, deskripsi ≤30 kata, mode dari tiga opsi.

In [ ]:
# ambil token pertama huruf/angka lalu kapitalisasi (memastikan satu kata bersih)
def sanitize_genre(text: str) -> str:
    tokens = re.findall(r"[A-Za-z0-9\-\+]+", str(text))
    return tokens[0].capitalize() if tokens else "Unknown"

# memotong menjadi <=30 kata dan tambahkan "..." jika kepanjangan
def sanitize_description(text: str, max_words: int = 30) -> str:
    words = str(text).strip().split()
    if len(words) <= max_words:
        return " ".join(words)
    return " ".join(words[:max_words]).rstrip(".,") + "..."

# normalisasi bentuk teks jadi salah satu dari tiga pilihan
def sanitize_mode(text: str) -> str:
    t = str(text).lower()
    if "single" in t and "multi" not in t:
        return "Singleplayer"
    if "multi" in t and "single" not in t:
        return "Multiplayer"
    if "both" in t:
        return "Both"
    # fallback simple mapping
    if "coop" in t or "co-op" in t:
        return "Multiplayer"
    return "Unknown"

Bagian 4 - Alur utama (main): baca csv, loop, panggil API, simpan

Gabungkan semua ke alur utama yang menerima arg --mock supaya mudah

In [ ]:
# membaca csv dan memproses tiap judul
def process_file(input_csv, output_csv, use_mock=True):
    df = pd.read_csv(input_csv)
    if "game_title" not in df.columns:
        raise SystemExit("Input CSV must have a 'game_title' column.")
    genres, descs, modes = [], [], []
    for i, title in enumerate(df["game_title"].fillna("").astype(str), start=1):
        print(f"[{i}/{len(df)}] Processing: {title}")

        # siapkan prompt
        p_genre = PROMPT_GENRE.format(title=title)
        p_desc  = PROMPT_DESC.format(title=title)
        p_mode  = PROMPT_MODE.format(title=title)

        # menentukan fungsi mana yang dipanggil: mock atau real
        if use_mock:
            raw_g = call_api_mock(p_genre, "genre")
            raw_d = call_api_mock(p_desc, "desc")
            raw_m = call_api_mock(p_mode, "mode")
        else:
            raw_g = call_api_real(p_genre)
            raw_d = call_api_real(p_desc)
            raw_m = call_api_real(p_mode)

        # rapikan hasil
        g = sanitize_genre(raw_g)
        d = sanitize_description(raw_d)
        m = sanitize_mode(raw_m)

        genres.append(g); descs.append(d); modes.append(m)

        # jeda kecil supaya tidak ngebomb endpoint kalau real
        # untuk memberi jeda antar request (penting saat pakai API nyata agar tidak kena rate limit)
        time.sleep(0.2)

    # nambah kolom baru
    df["genre"] = genres
    df["short_description"] = descs
    df["player_mode"] = modes

    # simpan csv output
    df.to_csv(output_csv, index=False)
    print("Saved:", output_csv)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--mock", action="store_true", help="Run in mock mode (no real API calls).")
    parser.add_argument("--input", default=INPUT_CSV, help="Input CSV file.")
    parser.add_argument("--output", default=OUTPUT_CSV, help="Output CSV file.")
    args = parser.parse_args()
    process_file(args.input, args.output, use_mock=args.mock)


usage: colab_kernel_launcher.py [-h] [--mock] [--input INPUT]
                                [--output OUTPUT]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-cb5eabe6-b330-446c-930a-60f8a77d0a2c.json


SystemExit: 2

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
